In [1]:
import copy, os
import math
import numpy as np
import pandas as pd
import scipy.optimize as opt
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

### Data for Hill fitting

In [2]:
drug_block_df = pd.read_csv( "./Hill_fitting/data/drug_block.csv")
drug_block_df.head()

,drug,conc,units,channel,block,pacing
0,amiodarone,0.8,nM,ICaL,0.0,0.1
1,amiodarone,0.8,nM,ICaL,1.9,0.1
2,amiodarone,0.8,nM,ICaL,0.3,0.1
3,amiodarone,80.0,nM,ICaL,17.4,0.1
4,amiodarone,80.0,nM,ICaL,12.9,0.1


In [3]:
hill_result_dir = "./Hill_fitting/results"
drug_li = os.listdir(hill_result_dir)
drug_li               

['bepridil',
 'chlorpromazine',
 'cisapride',
 'diltiazem',
 'dofetilide',
 'mexiletine',
 'ondansetron',
 'quinidine',
 'ranolazine',
 'sotalol',
 'terfenadine',
 'verapamil']

In [4]:
def get_Gfc(conc, IC50, h):
    return 1.0+(conc/10**np.log10(IC50))**h

In [5]:
ions = ["INa", "INaL", "Ito", "ICaL", "IKs", "IK1", 'hERG',]
optimized = True

for ion in ions :
    
    gfc_li = []    
    
    for drug in drug_li : 
        
        drug_ion_df = drug_block_df[(drug_block_df['drug']==drug) & (drug_block_df['channel']==ion)]        
                 
        IC50_df = pd.read_csv(f"{hill_result_dir}/{drug}/IC50_optimal.csv")    
        IC50 = IC50_df[f"{ion}_IC50"].item()
        h =  IC50_df[f"{ion}_h"].item()    
        if math.isnan(IC50) or math.isnan(h):        
            continue        
        gfcs = get_Gfc(drug_ion_df['conc'], IC50, h)                                        
                
        if optimized :
            if ion=="INaL":            
                gfcs = gfcs/2.661
            elif ion=="ICaL":
                gfcs = gfcs/1.007
            elif ion=="hERG":
                gfcs = np.full(gfcs.shape, 1.1)                
                gfcs = gfcs/1.114
            elif ion=="IKs":
                gfcs = gfcs/1.870
            elif ion=="Ik1":
                gfcs = gfcs/1.698
                       
        
        gfc_li.append(gfcs.min())
        gfc_li.append(gfcs.max())
        
    gfc_li = np.array(gfc_li)    
    print(f"Gfc of {ion} - min: {gfc_li.min()}  |  max: {gfc_li.max()}")

Gfc of INa - min: 1.0000610032568693  |  max: 6.335927561138344
Gfc of INaL - min: 0.37665083057147214  |  max: 1.4094220816010146
Gfc of Ito - min: 1.0000000019929693  |  max: 2.7518842312791723
Gfc of ICaL - min: 0.9945895580245948  |  max: 29.777326113370542
Gfc of IKs - min: 0.5351896228942729  |  max: 1.1454125735490408
Gfc of IK1 - min: 1.0015305275392006  |  max: 2.0894732182411744
Gfc of hERG - min: 0.9874326750448833  |  max: 0.9874326750448833


### Data for hERG fitting 

In [6]:
hERG_results_dir = "./hERG_fitting/results"
hERG_results_drug_list = os.listdir(hERG_results_dir)
hERG_results_drug_list
# drug_block_df = pd.read_csv( "./hERG_fitting/results/drug_block.csv")
# drug_block_df.head()

['bepridil',
 'chlorpromazine',
 'cisapride',
 'diltiazem',
 'dofetilide',
 'mexiletine',
 'ondansetron',
 'quinidine',
 'ranolazine',
 'sotalol',
 'terfenadine',
 'verapamil']

In [84]:
for i, drug in enumerate(hERG_results_drug_list):    
    temp_df1 = pd.read_csv( os.path.join(os.path.join(hERG_results_dir, drug), 'pars.txt'), sep=' ', names=[drug] )
    temp_df2 = pd.read_csv( os.path.join(os.path.join(hERG_results_dir, drug), 'boot_pars.csv') )     
    if i==0:
        hERG_pars_df = temp_df1
        hERG_boot_pars_df = temp_df2        
    else:        
        hERG_pars_df = pd.merge(hERG_pars_df, temp_df1, left_index=True, right_index=True) 
        hERG_boot_pars_df = pd.concat([hERG_boot_pars_df, temp_df2])
        
hERG_boot_pars_df.head()

,Kmax,Ku,n,halfmax,Vhalf,slope
0,712100.0,0.000179,0.9343,24420000.0,-14.66,0.02916
1,26750000.0,0.000148,0.8436,413000000.0,-67.50,0.06477
2,801300.0,0.000171,1.0160,24850000.0,-72.57,0.03225
3,9883000.0,0.000137,0.9805,378600000.0,-40.66,0.02610
4,148700.0,0.000177,0.8687,2588000.0,-67.27,0.05746


In [110]:
print( "===== hERG pars ==================================" )
print( f"Kmax     min: %f  | max: %f"%(hERG_pars_df.loc['Kmax'].min(), hERG_pars_df.loc['Kmax'].max()) )
print( f"Ku       min: %f  | max: %f"%(hERG_pars_df.loc['Ku'].min(), hERG_pars_df.loc['Ku'].max()) ) 
print( f"n        min: %f  | max: %f"%(hERG_pars_df.loc['n'].min(), hERG_pars_df.loc['n'].max()) )
print( f"halfmax  min: %f  | max: %f"%(hERG_pars_df.loc['halfmax'].min(), hERG_pars_df.loc['halfmax'].max()) )
print( f"Vhalf    min: %f  | max: %f"%(hERG_pars_df.loc['Vhalf'].min(), hERG_pars_df.loc['Vhalf'].max()) )
print( "="*50)

===== hERG pars ==================================
Kmax     min: 10.220000  | max: 5594000.000000
Ku       min: 0.000018  | max: 0.282000
n        min: 0.650200  | max: 1.139000
halfmax  min: 42.320000  | max: 667700000.000000
Vhalf    min: -167.400000  | max: -1.000000


In [111]:
print( "===== hERG boot pars =============================" )
print( f"Kmax     min: %f  | max: %f"%(hERG_boot_pars_df['Kmax'].min(), hERG_boot_pars_df['Kmax'].max()) )
print( f"Ku       min: %f  | max: %f"%(hERG_boot_pars_df['Ku'].min(), hERG_boot_pars_df['Ku'].max()) ) 
print( f"n        min: %f  | max: %f"%(hERG_boot_pars_df['n'].min(), hERG_boot_pars_df['n'].max()) )
print( f"halfmax  min: %f  | max: %f"%(hERG_boot_pars_df['halfmax'].min(), hERG_boot_pars_df['halfmax'].max()) )
print( f"Vhalf    min: %f  | max: %f"%(hERG_boot_pars_df['Vhalf'].min(), hERG_boot_pars_df['Vhalf'].max()) )
print( "="*50)

===== hERG boot pars =============================
Kmax     min: 2.847000  | max: 74530000.000000
Ku       min: 0.000009  | max: 1.000000
n        min: 0.500000  | max: 1.812000
halfmax  min: 9.636000  | max: 1000000000.000000
Vhalf    min: -199.700000  | max: -1.000000


In [112]:
Kmax = [2.5, 75000000]
Ku = [0, 1]
n = [0.5, 2]
halfmax = [9, 1000000000]
Vhalf = [-200, -1]